# Exercise 1: Schema on Read

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import matplotlib

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
dfLog = spark.read.text("data/NASA_access_log_Jul95.gz")

# Load the dataset

In [4]:
#Data Source: http://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz
dfLog = spark.read.text("data/NASA_access_log_Jul95.gz")

# Quick inspection of  the data set

In [5]:
# see the schema
dfLog.printSchema()

root
 |-- value: string (nullable = true)



In [6]:
# number of lines
dfLog.count()

1891715

In [7]:
#what's in there? 
dfLog.show(5)

+--------------------+
|               value|
+--------------------+
|199.72.81.55 - - ...|
|unicomp6.unicomp....|
|199.120.110.21 - ...|
|burger.letters.co...|
|199.120.110.21 - ...|
+--------------------+
only showing top 5 rows



In [9]:
#a better show?
dfLog.show(5, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                  |
+-----------------------------------------------------------------------------------------------------------------------+
|199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245                                 |
|unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985                      |
|199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085   |
|burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0               |
|199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179|
+-----------------------

In [10]:
#pandas to the rescue
pd.set_option('max_colwidth', 200)
dfLog.limit(5).toPandas()

value
0                                   199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245
1                        unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985
2     199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085
3                 burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0
4  199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179

# Let' try simple parsing with split

In [16]:
from pyspark.sql.functions import split
# Todo
dfLog.withColumn("split_value", split("value", ' ')).select("split_value").limit(5).toPandas()

split_value
0                                   [199.72.81.55, -, -, [01/Jul/1995:00:00:01, -0400], "GET, /history/apollo/, HTTP/1.0", 200, 6245]
1                        [unicomp6.unicomp.net, -, -, [01/Jul/1995:00:00:06, -0400], "GET, /shuttle/countdown/, HTTP/1.0", 200, 3985]
2     [199.120.110.21, -, -, [01/Jul/1995:00:00:09, -0400], "GET, /shuttle/missions/sts-73/mission-sts-73.html, HTTP/1.0", 200, 4085]
3                 [burger.letters.com, -, -, [01/Jul/1995:00:00:11, -0400], "GET, /shuttle/countdown/liftoff.html, HTTP/1.0", 304, 0]
4  [199.120.110.21, -, -, [01/Jul/1995:00:00:11, -0400], "GET, /shuttle/missions/sts-73/sts-73-patch-small.gif, HTTP/1.0", 200, 4179]

# Second attempt, let's build a custom parsing UDF 

In [4]:
from pyspark.sql.functions import udf
from pyspark.sql.types import MapType, StringType

@udf(MapType(StringType(), StringType()))
def parseUDF(line):
    # Todo
    import re
    PATTERN = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+)\s*(\S+)" (\d{3}) (\S+)'
    match = re.search(PATTERN, line)
    col_names = ["host", "client_identd","user_id", "date_time", "method", "endpoint", "protocol", "response_code", "content_size"]
    if match is None:
        return(line, 0)
    size_field = match.group(9)
    if size_field == '_':
        size = 0
    else:
        size = size_field
    
    col_values = [match.group(1), match.group(2), match.group(3), match.group(4), match.group(5),
                 match.group(6), match.group(7), int(match.group(8)), size]
    
    return dict(zip(col_names, col_values))

dfParsed = dfLog.withColumn("parsed", parseUDF("value"))
dfParsed.limit(5).toPandas()

value  \
0  199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] ...   
1  unicomp6.unicomp.net - - [01/Jul/1995:00:00:06...   
2  199.120.110.21 - - [01/Jul/1995:00:00:09 -0400...   
3  burger.letters.com - - [01/Jul/1995:00:00:11 -...   
4  199.120.110.21 - - [01/Jul/1995:00:00:11 -0400...   

                                              parsed  
0  {'response_code': '200', 'protocol': 'HTTP/1.0...  
1  {'response_code': '200', 'protocol': 'HTTP/1.0...  
2  {'response_code': '200', 'protocol': 'HTTP/1.0...  
3  {'response_code': '304', 'protocol': 'HTTP/1.0...  
4  {'response_code': '200', 'protocol': 'HTTP/1.0...

In [ ]:
#Let's start from the beginning
# Todo

In [25]:
dfParsed.printSchema()

root
 |-- value: string (nullable = true)
 |-- parsed: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



# Third attempt, let's fix our UDF

In [ ]:
#from pyspark.sql.functions import udf # already imported
from pyspark.sql.types import MapType, StringType

@udf(MapType(StringType(),StringType()))
def parseUDFbetter(line):
    # Todo

In [ ]:
#Let's start from the beginning
# Todo

In [ ]:
#Let's start from the beginning
# Todo

In [ ]:
#Bingo!! we'got a column of type map with the fields parsed
dfParsed.printSchema()

In [26]:
dfParsed.select("parsed").limit(10).toPandas()

parsed
0  {'response_code': '200', 'protocol': 'HTTP/1.0', 'endpoint': '/history/apollo/', 'content_size': '6245', 'method': 'GET', 'date_time': '01/Jul/1995:00:00:01 -0400', 'user_id': '-', 'host': '199.72...
1  {'response_code': '200', 'protocol': 'HTTP/1.0', 'endpoint': '/shuttle/countdown/', 'content_size': '3985', 'method': 'GET', 'date_time': '01/Jul/1995:00:00:06 -0400', 'user_id': '-', 'host': 'uni...
2  {'response_code': '200', 'protocol': 'HTTP/1.0', 'endpoint': '/shuttle/missions/sts-73/mission-sts-73.html', 'content_size': '4085', 'method': 'GET', 'date_time': '01/Jul/1995:00:00:09 -0400', 'us...
3  {'response_code': '304', 'protocol': 'HTTP/1.0', 'endpoint': '/shuttle/countdown/liftoff.html', 'content_size': '0', 'method': 'GET', 'date_time': '01/Jul/1995:00:00:11 -0400', 'user_id': '-', 'ho...
4  {'response_code': '200', 'protocol': 'HTTP/1.0', 'endpoint': '/shuttle/missions/sts-73/sts-73-patch-small.gif', 'content_size': '4179', 'method': 'GET', 'date_time': '01/Jul/1995:00:00:11 -0400', ...
5  {'response_code': '304', 'protocol': 'HTTP/1.0', 'endpoint': '/images/NASA-logosmall.gif', 'content_size': '0', 'method': 'GET', 'date_time': '01/Jul/1995:00:00:12 -0400', 'user_id': '-', 'host': ...
6  {'response_code': '200', 'protocol': 'HTTP/1.0', 'endpoint': '/shuttle/countdown/video/livevideo.gif', 'content_size': '0', 'method': 'GET', 'date_time': '01/Jul/1995:00:00:12 -0400', 'user_id': '...
7  {'response_code': '200', 'protocol': 'HTTP/1.0', 'endpoint': '/shuttle/countdown/countdown.html', 'content_size': '3985', 'method': 'GET', 'date_time': '01/Jul/1995:00:00:12 -0400', 'user_id': '-'...
8  {'response_code': '200', 'protocol': 'HTTP/1.0', 'endpoint': '/shuttle/countdown/', 'content_size': '3985', 'method': 'GET', 'date_time': '01/Jul/1995:00:00:13 -0400', 'user_id': '-', 'host': 'd10...
9  {'response_code': '200', 'protocol': 'HTTP/1.0', 'endpoint': '/', 'content_size': '7074', 'method': 'GET', 'date_time': '01/Jul/1995:00:00:13 -0400', 'user_id': '-', 'host': '129.94.144.152', 'cli...

In [27]:
dfParsed.selectExpr("parsed['host'] as host").limit(5).show()

+--------------------+
|                host|
+--------------------+
|        199.72.81.55|
|unicomp6.unicomp.net|
|      199.120.110.21|
|  burger.letters.com|
|      199.120.110.21|
+--------------------+



# Let's build separate columns

In [28]:
dfParsed.selectExpr("parsed['host'] as host").limit(5).show(5)

+--------------------+
|                host|
+--------------------+
|        199.72.81.55|
|unicomp6.unicomp.net|
|      199.120.110.21|
|  burger.letters.com|
|      199.120.110.21|
+--------------------+



In [29]:
dfParsed.selectExpr(["parsed['host']", "parsed['date_time']"]).show(5)

+--------------------+--------------------+
|        parsed[host]|   parsed[date_time]|
+--------------------+--------------------+
|        199.72.81.55|01/Jul/1995:00:00...|
|unicomp6.unicomp.net|01/Jul/1995:00:00...|
|      199.120.110.21|01/Jul/1995:00:00...|
|  burger.letters.com|01/Jul/1995:00:00...|
|      199.120.110.21|01/Jul/1995:00:00...|
+--------------------+--------------------+
only showing top 5 rows



In [5]:
fields = ["host", "client_identd","user_id", "date_time", "method", "endpoint", "protocol", "response_code", "content_size"]
exprs = [f"parsed['{field}'] as {field}" for field in fields]
exprs

["parsed['host'] as host",
 "parsed['client_identd'] as client_identd",
 "parsed['user_id'] as user_id",
 "parsed['date_time'] as date_time",
 "parsed['method'] as method",
 "parsed['endpoint'] as endpoint",
 "parsed['protocol'] as protocol",
 "parsed['response_code'] as response_code",
 "parsed['content_size'] as content_size"]

In [6]:
dfClean = dfParsed.selectExpr(*exprs)
dfClean.limit(5).toPandas()

host client_identd user_id                   date_time  \
0          199.72.81.55             -       -  01/Jul/1995:00:00:01 -0400   
1  unicomp6.unicomp.net             -       -  01/Jul/1995:00:00:06 -0400   
2        199.120.110.21             -       -  01/Jul/1995:00:00:09 -0400   
3    burger.letters.com             -       -  01/Jul/1995:00:00:11 -0400   
4        199.120.110.21             -       -  01/Jul/1995:00:00:11 -0400   

  method                                         endpoint  protocol  \
0    GET                                 /history/apollo/  HTTP/1.0   
1    GET                              /shuttle/countdown/  HTTP/1.0   
2    GET     /shuttle/missions/sts-73/mission-sts-73.html  HTTP/1.0   
3    GET                  /shuttle/countdown/liftoff.html  HTTP/1.0   
4    GET  /shuttle/missions/sts-73/sts-73-patch-small.gif  HTTP/1.0   

  response_code content_size  
0           200         6245  
1           200         3985  
2           200         4085  
3           304            0  
4           200         4179

## Popular hosts

In [42]:
from pyspark.sql.functions import desc
# Todo
dfClean.createOrReplaceTempView("dfCleanTable")

spark.sql("""
    SELECT host, count(host) as count_host
    FROM dfCleanTable
    GROUP BY host
    ORDER BY count_host DESC
    LIMIT 5
""").show()

+--------------------+----------+
|                host|count_host|
+--------------------+----------+
|piweba3y.prodigy.com|     17572|
|piweba4y.prodigy.com|     11591|
|piweba1y.prodigy.com|      9868|
|  alyssa.prodigy.com|      7852|
| siltb10.orl.mmc.com|      7573|
+--------------------+----------+



## Popular content

In [44]:
from pyspark.sql.functions import desc
# Todo
spark.sql("""
    SELECT endpoint, content_size
    FROM dfCleanTable
    ORDER BY content_size DESC
""").limit(5).show()

+--------------------+------------+
|            endpoint|content_size|
+--------------------+------------+
|/images/cdrom-1-9...|       99981|
|/shuttle/missions...|      999424|
|/shuttle/missions...|      999424|
|/history/apollo/a...|       99942|
|/history/apollo/a...|       99942|
+--------------------+------------+



## Large Files

In [ ]:
dfClean.createOrReplaceTempView("cleanlog")
spark.sql("""
select endpoint, content_size
from cleanlog 
order by content_size desc
""").limit(10).toPandas()

In [45]:
from pyspark.sql.functions import expr
dfCleanTyped = dfClean.withColumn("contentSizeBytes", expr("CAST(content_size as int)"))

In [47]:
dfCleanTyped.createOrReplaceTempView("cleantypedlog")
spark.sql("""
select endpoint, content_size
from cleantypedlog 
order by contentSizeBytes desc
""").limit(10).toPandas()

endpoint content_size
0          /shuttle/countdown/video/livevideo.jpeg      6823936
1            /statistics/1995/bkup/Mar95_full.html      3155499
2            /statistics/1995/bkup/Mar95_full.html      3155499
3            /statistics/1995/bkup/Mar95_full.html      3155499
4            /statistics/1995/bkup/Mar95_full.html      3155499
5            /statistics/1995/bkup/Mar95_full.html      3155499
6            /statistics/1995/bkup/Mar95_full.html      3155499
7            /statistics/1995/bkup/Mar95_full.html      3155499
8  /statistics/1995/Jun/Jun95_reverse_domains.html      2973350
9  /statistics/1995/Jun/Jun95_reverse_domains.html      2973350

In [ ]:
# Left for you, clean the date column :)
# 1- Create a udf that parses that weird format,
# 2- Create a new column with a data tiem string that spark would understand
# 3- Add a new date-time column properly typed
# 4- Print your schema

In [141]:
dfDate = dfClean.select('date_time').dropna()
dfDate.limit(5).show(truncate = False)

+--------------------------+
|date_time                 |
+--------------------------+
|01/Jul/1995:00:00:01 -0400|
|01/Jul/1995:00:00:06 -0400|
|01/Jul/1995:00:00:09 -0400|
|01/Jul/1995:00:00:11 -0400|
|01/Jul/1995:00:00:11 -0400|
+--------------------------+



In [137]:
import datetime
datetime.datetime(year=2012, month=12, day=9, hour=0, minute=0, second=10)

datetime.datetime(2012, 12, 9, 0, 0, 10)

In [146]:
from pyspark.sql.functions import udf
from pyspark.sql.types import MapType, DateType, DatetimeConverter, StringType, TimestampType

@udf(TimestampType()) #(MapType(StringType(), StringType()))
def parseDateTime(line):
    import re, datetime
    line = line.split(' ')[0]
    line = re.sub("[^a-zA-Z0-9]", "", line)
    PATTERN = "^(\d{2})(\w+)(\d{4})(\d{2})(\d{2})(\d{2})"
    match = re.search(PATTERN, line)
    cols = ['day', 'month', 'year', 'hour', 'minute', 'second']
    month = match.group(2)
    month  = datetime.datetime.strptime(month, "%b").month
    #values = [match.group(1), month, match.group(3), match.group(4), match.group(5), match.group(6)]
    #dt = dict(zip(cols, values))
    dt = datetime.datetime(day=int(match.group(1)),
                           month=int(month),
                           year=int(match.group(3)),
                           hour=int(match.group(4)),
                           minute=int(match.group(5)),
                           second=int(match.group(6))
                          )
    return dt

dfDateParsed = dfDate.withColumn("dateTimeParsed", parseDateTime('date_time'))


In [147]:
#cols = ['day', 'month', 'year', 'hour', 'minute', 'second']
#exprs = [f"dateTimeParsed['{col}'] as {col}" for col in cols]
#dfDateTime = dfDateParsed.selectExpr(*exprs)

In [149]:
dfDateParsed.printSchema()

root
 |-- date_time: string (nullable = true)
 |-- dateTimeParsed: timestamp (nullable = true)



In [128]:
dfDateTime.groupBy('month').count().show()

+-----+-------+
|month|  count|
+-----+-------+
|    7|1890638|
+-----+-------+



In [21]:
dfDateTime.createOrReplaceTempView("dfDateTimeTable")

In [76]:
dfd.printSchema()

root
 |-- date_time: string (nullable = true)
 |-- dateTimeParsed: string (nullable = true)



In [63]:
import datetime